# Imports 

In [47]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import re

# Data Collection

In [4]:
# Parameters
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# URL
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

# Request to URL
page = requests.get( url, headers=headers )

#Beatiful soup object
soup = BeautifulSoup( page.text, 'html.parser' )

#=========================Product data===========================================
products = soup.find('ul',class_='products-listing small')
product_list = products.find_all( 'article', class_='hm-product-item')


# product id
product_id = [p.get( 'data-articlecode' ) for p in product_list]

# product category
product_category = [p.get( 'data-category' ) for p in product_list]

# product name 
product_list = products.find_all( 'a', class_='link' )
product_name = [p.get_text() for p in product_list]

# price
product_list = products.find_all( 'span', class_='price regular' )
product_price = [p.get_text() for p in product_list]


#Tabela
data = pd.DataFrame( [product_id, product_category, product_name, product_price] ).T
data.columns = ['product_id', 'product_category', 'product_name','product_price']




,product_id,product_category,product_name,product_price,scrapy_datetime
0,0985197006,men_jeans_slim,Slim Jeans,$ 19.99,2021-07-24 15:12:55
1,0985159002,men_jeans_skinny,Skinny Jeans,$ 19.99,2021-07-24 15:12:55
2,1004476004,,Freefit® Slim Jeans,$ 49.99,2021-07-24 15:12:55
3,0986779004,,Relaxed Tapered Pull-on Jeans,$ 29.99,2021-07-24 15:12:55
4,0986779003,,Relaxed Tapered Pull-on Jeans,$ 29.99,2021-07-24 15:12:55


# Data Collection by Product

In [18]:
# empty dataframe

df_compositions = pd.DataFrame()

#unique columns for all products
aux = []
df_pattern = pd.DataFrame(columns = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size'])
for i in range(len(data)):

      # API Requests
      url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i,'product_id'] + '.html'
      #url = 'https://www2.hm.com/en_us/productpage.' + '0985197006' + '.html'
      print('Product: {}'.format(url))
      page = requests.get( url, headers=headers )
           
#     # Beautiful Soup object
      soup = BeautifulSoup( page.text, 'html.parser' )

#     # ==================== color name =================================

      product_list = soup.find_all( 'a', class_='filter-option miniature active' ) + soup.find_all( 'a', class_='filter-option miniature' )
      color_name = [p.get( 'data-color' ) for p in product_list]
#     # product id

      product_id = [p.get( 'data-articlecode' ) for p in product_list]
      df_color = pd.DataFrame( [product_id, color_name] ).T
      df_color.columns = ['product_id', 'color_name']
      
      for j in range(len(df_color)):
        
            # API Requests
            url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j,'product_id'] + '.html'
            print('Color: {}'.format(url))
      
            page = requests.get( url, headers=headers )

#     #     # Beautiful Soup object
            soup = BeautifulSoup( page.text, 'html.parser' )
    
            # ========================Product name===========================
            product_name = soup.find_all('h1', class_='primary product-item-headline')
            product_name = product_name[0].get_text()
            
            # ========================Product price===========================
            product_price = soup.find_all('div', class_='primary-row product-item-price')
            product_price = re.findall(r'\d+\.?\d+', product_price[0].get_text())[0]
            #print(product_price)
 
  

    #     # ==================== composition =================================

            product_composition_list = soup.find_all( 'div',class_='pdp-description-list-item' )
            product_composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]

#     #     # reaname dataframe
            df_composition = pd.DataFrame( product_composition ).T
            df_composition.columns = df_composition.iloc[0]


#           # delete first row
            df_composition = df_composition.iloc[1:].fillna( method='ffill' )

#           # Remove pocket lining, shell and lining

            df_composition['Composition'] = df_composition['Composition'].replace('Pocket lining: ', '', regex=True) 
            df_composition['Composition'] = df_composition['Composition'].replace('Shell: ', '', regex=True) 
            df_composition['Composition'] = df_composition['Composition'].replace('Lining: ', '', regex=True) 



#     #     #Garante the same number of columns
            df_composition = pd.concat([df_pattern,df_composition], axis = 0)

#           # Rename coluns
            df_composition.columns = ['product_id','composition', 'fit','product_safety','size']
            df_composition['product_name'] = product_name
            df_composition['product_price'] = product_price
            
#           # Keep new coluns if it shows up

            aux = aux + df_composition.columns.tolist()


#     #     # merge data color + decomposition
            df_composition = pd.merge(df_composition, df_color, how='left', on='product_id' )

#     #     #all products
            df_compositions = pd.concat([df_compositions, df_composition], axis=0)
            #print(df_composition)
          
 
          
# generate style id + color id
df_compositions['style_id'] = df_compositions['product_id'].apply( lambda x: x[:-3] )
df_compositions['color_id'] = df_compositions['product_id'].apply( lambda x: x[-3:] )


# scrapy datetime
df_composition['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S' )



# # Join Showroom data + details
# data['style_id'] = data['product_id'].apply( lambda x: x[:-3])
# data['color_id'] = data['product_id'].apply( lambda x: x[-3:])
# data_raw = pd.merge(data, df_details[['style_id','color_name','Fit','Composition','Size','Product safety']], 
#                     how='left', on='style_id')

Product: https://www2.hm.com/en_us/productpage.0985197006.html
Color: https://www2.hm.com/en_us/productpage.0985197006.html
Color: https://www2.hm.com/en_us/productpage.0985197001.html
Color: https://www2.hm.com/en_us/productpage.0985197002.html
Color: https://www2.hm.com/en_us/productpage.0985197003.html
Color: https://www2.hm.com/en_us/productpage.0985197004.html
Color: https://www2.hm.com/en_us/productpage.0985197005.html
Product: https://www2.hm.com/en_us/productpage.0985159002.html
Color: https://www2.hm.com/en_us/productpage.0985159002.html
Color: https://www2.hm.com/en_us/productpage.0985159001.html
Color: https://www2.hm.com/en_us/productpage.0985159003.html
Color: https://www2.hm.com/en_us/productpage.0985159004.html
Color: https://www2.hm.com/en_us/productpage.0985159005.html
Color: https://www2.hm.com/en_us/productpage.0985159006.html
Product: https://www2.hm.com/en_us/productpage.1004476004.html
Color: https://www2.hm.com/en_us/productpage.1004476004.html
Color: https://www

Color: https://www2.hm.com/en_us/productpage.0985159006.html
Product: https://www2.hm.com/en_us/productpage.0985197003.html
Color: https://www2.hm.com/en_us/productpage.0985197003.html
Color: https://www2.hm.com/en_us/productpage.0985197001.html
Color: https://www2.hm.com/en_us/productpage.0985197002.html
Color: https://www2.hm.com/en_us/productpage.0985197004.html
Color: https://www2.hm.com/en_us/productpage.0985197005.html
Color: https://www2.hm.com/en_us/productpage.0985197006.html
Product: https://www2.hm.com/en_us/productpage.0985197001.html
Color: https://www2.hm.com/en_us/productpage.0985197001.html
Color: https://www2.hm.com/en_us/productpage.0985197002.html
Color: https://www2.hm.com/en_us/productpage.0985197003.html
Color: https://www2.hm.com/en_us/productpage.0985197004.html
Color: https://www2.hm.com/en_us/productpage.0985197005.html
Color: https://www2.hm.com/en_us/productpage.0985197006.html
Product: https://www2.hm.com/en_us/productpage.0985197002.html
Color: https://www

Color: https://www2.hm.com/en_us/productpage.0974202001.html
Color: https://www2.hm.com/en_us/productpage.0974202003.html
Product: https://www2.hm.com/en_us/productpage.0636207001.html
Color: https://www2.hm.com/en_us/productpage.0636207001.html
Color: https://www2.hm.com/en_us/productpage.0636207002.html
Color: https://www2.hm.com/en_us/productpage.0636207004.html
Color: https://www2.hm.com/en_us/productpage.0636207005.html
Color: https://www2.hm.com/en_us/productpage.0636207006.html
Color: https://www2.hm.com/en_us/productpage.0636207010.html
Color: https://www2.hm.com/en_us/productpage.0636207011.html
Color: https://www2.hm.com/en_us/productpage.0636207014.html
Color: https://www2.hm.com/en_us/productpage.0636207015.html
Color: https://www2.hm.com/en_us/productpage.0636207017.html
Color: https://www2.hm.com/en_us/productpage.0636207019.html
Product: https://www2.hm.com/en_us/productpage.0927964002.html
Color: https://www2.hm.com/en_us/productpage.0927964002.html
Color: https://www2.

In [20]:
df_compositions

,product_id,composition,fit,product_safety,size,product_name,product_price,color_name,style_id,color_id
0,0985197006,"Polyester 65%, Cotton 35%",Slim fit,NaN,"The model is 187cm/6'2"" and wears a size 31/32",\n\t\t\t\t\t\t\t Slim Jeans,19.99,Light denim blue,0985197,006
1,0985197006,"Cotton 99%, Elastane 1%",Slim fit,NaN,"The model is 187cm/6'2"" and wears a size 31/32",\n\t\t\t\t\t\t\t Slim Jeans,19.99,Light denim blue,0985197,006
0,0985197001,Cotton 100%,Slim fit,NaN,"The model is 189cm/6'2"" and wears a size 32/32",\n\t\t\t\t\t\t\t Slim Jeans,19.99,Black,0985197,001
1,0985197001,"Cotton 98%, Elastane 2%",Slim fit,NaN,"The model is 189cm/6'2"" and wears a size 32/32",\n\t\t\t\t\t\t\t Slim Jeans,19.99,Black,0985197,001
0,0985197002,"Cotton 99%, Elastane 1%",Slim fit,NaN,"The model is 182cm/6'0"" and wears a size 31/32",\n\t\t\t\t\t\t\t Slim Jeans,19.99,Midnight blue,0985197,002
...,...,...,...,...,...,...,...,...,...,...
0,0751994017,"Cotton 98%, Elastane 2%",Slim fit,NaN,NaN,\n\t\t\t\t\t\t\t Slim Jeans,16.99,Dark denim blue,0751994,017
0,0751994024,"Cotton 98%, Elastane 2%",Slim fit,NaN,"The model is 187cm/6'2"" and wears a size 31/32",\n\t\t\t\t\t\t\t Slim Jeans,29.99,Black denim,0751994,024
0,0751994031,"Cotton 99%, Elastane 1%",Slim fit,NaN,"The model is 187cm/6'2"" and wears a size 31/32",\n\t\t\t\t\t\t\t Slim Jeans,29.99,Light blue,0751994,031
0,0971061002,"Cotton 99%, Elastane 1%",Slim fit,NaN,NaN,\n\t\t\t\t\t\t\t Slim Tapered Cropped Jeans,29.99,Light denim blue,0971061,002


# Data Cleaning

In [77]:
#Product Id
df_data = df_compositions.dropna(subset=['product_id'])

# # product name
df_data['product_name'] = df_data['product_name'].str.replace('\n','')
df_data['product_name'] = df_data['product_name'].str.replace('\t','')
df_data['product_name'] = df_data['product_name'].str.replace('  ','')
df_data['product_name'] = df_data['product_name'].str.replace(' ','_').str.lower()



# # # product price
#df_data['product_price'] = df_data['product_price'].apply( lambda x: x.replace( '$ ','' ) )
df_data['product_price'] = df_data['product_price'].astype(float)

# # # color name
df_data['color_name'] = df_data['color_name'].str.replace(' ','_').str.lower()
# # # fit
df_data['fit'] = df_data['fit'].apply( lambda x: x.replace( ' ', '_' ).lower() if pd.notnull( x ) else x )

# # # size number
df_data['size_number'] = df_data['size'].apply( lambda x: re.search( '\d{3}cm', x ).group(0) if pd.notnull( x ) else x )
df_data['size_number'] = df_data['size_number'].apply( lambda x: re.search( '\d+', x ).group(0) if pd.notnull( x ) else x )

# # # size model
df_data['size_model'] = df_data['size'].str.extract( '(\d+/\\d+)' )

# # # break composition by comma
df1 = df_data['composition'].str.split( ',', expand=True ).reset_index(drop=True)

# # # cotton | polyester | elastano | elasterell
df_ref = pd.DataFrame( index=np.arange( len( df_data ) ),columns=['cotton','polyester', 'elastane', 'elasterell'] )

# # ================================composition==============================================================

# ===================== cotton=============================================
df_cotton_0 = df1.loc[df1[0].str.contains('Cotton',na=True),0]
df_cotton_0.name = 'cotton'

df_cotton_1 = df1.loc[df1[1].str.contains('Cotton',na=True),1]
df_cotton_1.name = 'cotton'

#Combine
df_cotton = df_cotton_0.combine_first(df_cotton_1)
df_ref = pd.concat( [df_ref, df_cotton ], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last')]


# ==================== polyester============================================
df_polyester_0 = df1.loc[df1[0].str.contains( 'Polyester', na=True ), 0]
df_polyester_0.name = 'polyester'

df_polyester_1 = df1.loc[df1[1].str.contains( 'Polyester', na=True ), 1]
df_polyester_1.name = 'polyester'

#Combaine
df_polyester = df_polyester_0.combine_first(df_polyester_1)

df_ref = pd.concat( [df_ref, df_polyester], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last') ]


# ==================== elastano============================================
df_elastane_1 = df1.loc[df1[1].str.contains( 'Elastane', na=True ), 1]
df_elastane_1.name = 'elastane'

df_elastane_2 = df1.loc[df1[2].str.contains( 'Elastane', na=True ), 2]
df_elastane_2.name = 'elastane'

df_elastane_3 = df1.loc[df1[3].str.contains( 'Elastane', na=True ), 3]
df_elastane_3.name = 'elastane'

# combine 
df_elastane_c2 = df_elastane_1.combine_first(df_elastane_2 )
df_elastane = df_elastane_c2.combine_first(df_elastane_3 )

df_ref = pd.concat( [df_ref, df_elastane], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last') ]

# ==================== elasterell ============================================

df_elasterell = df1.loc[df1[1].str.contains( 'Elasterell', na=True ), 1]
df_elasterell.name = 'elasterell'
df_ref = pd.concat( [df_ref, df_elasterell], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last') ]


# Join of combine with product id
df_aux = pd.concat([df_data['product_id'].reset_index(drop=True), df_ref], axis=1)

# # # format composition data
df_aux['cotton']     = df_aux['cotton'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['polyester']  = df_aux['polyester'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['elastane']   = df_aux['elastane'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['elasterell'] = df_aux['elasterell'].apply( lambda x: int( re.search('\d+',x ).group(0) ) / 100 if pd.notnull( x ) else x )



# # # final join
df_aux = df_aux.groupby('product_id').max().reset_index().fillna(0)
df_data = pd.merge(df_data,df_aux, on='product_id', how='left')
# # data = pd.concat( [data, df_ref], axis=1 )


# # # Drop columns
df_data = df_data.drop( columns=['size','product_safety','composition'], axis=1)

#Drop duplicates
df_data = df_data.drop_duplicates()

#Export csv
df_data.to_csv('Base')

In [76]:
df_data

,product_id,fit,product_name,product_price,color_name,style_id,color_id,size_number,size_model,cotton,polyester,elastane,elasterell
0,0985197006,slim_fit,slim_jeans,19.99,light_denim_blue,0985197,006,187,31/32,0.99,0.65,0.01,0.0
2,0985197001,slim_fit,slim_jeans,19.99,black,0985197,001,189,32/32,1.00,0.00,0.02,0.0
4,0985197002,slim_fit,slim_jeans,19.99,midnight_blue,0985197,002,182,31/32,0.99,0.65,0.01,0.0
6,0985197003,slim_fit,slim_jeans,19.99,denim_blue,0985197,003,182,31/32,0.99,0.65,0.01,0.0
8,0985197004,slim_fit,slim_jeans,19.99,dark_denim_blue,0985197,004,NaN,NaN,1.00,0.00,0.01,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,0927964001,regular_fit,regular_tapered_crop_jeans,19.99,denim_blue,0927964,001,188,NaN,1.00,0.65,0.00,0.0
389,0927964003,regular_fit,regular_tapered_crop_jeans,19.99,dark_gray,0927964,003,NaN,NaN,1.00,0.65,0.00,0.0
391,0927964004,regular_fit,regular_tapered_crop_jeans,19.99,steel_blue,0927964,004,NaN,NaN,1.00,0.65,0.00,0.0
393,0927964005,regular_fit,regular_tapered_crop_jeans,10.99,white,0927964,005,187,NaN,1.00,0.00,0.00,0.0
